# Install

In [1]:
!pip install torch torchvision

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

# Import

In [2]:
import torch
import torch.nn.functional as F
from torch import nn

import torch.func as fc
from torch import Tensor

In [3]:
from typing import List, Optional
from typing import Dict, KeysView, ValuesView

In [4]:
import torchvision
import matplotlib.pyplot as plt
import torchvision.transforms as tt

In [5]:
import math
import time
from tqdm import tqdm
import copy
from functools import partial

# Constants

In [6]:
BATCH_SIZE =  64
SHUFFLE = True
EPOCHS = 50

# Model definitions

In [7]:
class NeuralNet(nn.Module):
    def __init__(
        self,
        input_size: int,
        hidden_sizes: List[int],
        output_size: int,
        activation_function: Optional[torch.nn.Module] = None,
    ):
        """Standard Fully-Connnected layers.

        Args:
            input_size (int): input size of the model.
            hidden_sizes (List[int]): a list of hidden sizes.
            output_size (int): The number of output classes.
            activation_function (Optional[torch.nn.Module], optional): the activation function for the hidden layers.
                Defaults to None.

        """
        super(NeuralNet, self).__init__()
        self.input_size = input_size
        self.hidden_sizes = hidden_sizes
        self.hidden_sizes.insert(0, input_size)
        for i in range(len(hidden_sizes) - 1):
            setattr(self, f"fc{i}", nn.Linear(hidden_sizes[i], hidden_sizes[i + 1]))
            setattr(self, f"act{i}", activation_function or nn.ReLU())
        self.out = nn.Linear(hidden_sizes[-1], output_size)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        for i in range(len(self.hidden_sizes) - 1):
            x = getattr(self, f"act{i}")(getattr(self, f"fc{i}")(x))
        x = self.out(x)
        return x


# Loss Definition

In [8]:
def cross_entropy(model: torch.nn.Module, x: torch.Tensor, t: torch.Tensor) -> torch.Tensor:
    """Cross-entropy loss. Given a pytorch model, it computes the cross-entropy loss.

    Args:
        model (torch.nn.Module): PyTorch model.
        x (torch.Tensor): Input tensor for the PyTorch model.
        t (torch.Tensor): Targets.

    Returns:
        torch.Tensor: Cross-entropy loss.
    """
    y = model(x)
    return F.cross_entropy(y, t)


def functional_cross_entropy(
    params: ValuesView,
    buffers: Dict[str, Tensor],
    names: KeysView,
    model: torch.nn.Module,
    x: torch.Tensor,
    t: torch.Tensor,
) -> torch.Tensor:
    """Functional cross-entropy loss. Given a pytorch model it computes the cross-entropy loss
    in a functional way.

    Args:
        params: Model parameters.
        buffers: Buffers of the model.
        names: Names of the parameters.
        model: A pytorch model.
        x (torch.Tensor): Input tensor for the PyTorch model.
        t (torch.Tensor): Targets.

    Returns:
        torch.Tensor: Cross-entropy loss.
    """
    y = fc.functional_call(model, ({k: v for k, v in zip(names, params)}, buffers), (x,))
    return F.cross_entropy(y, t)

# Utils

In [9]:
def exponential_lr_decay(step: int, k: float=1e-4):
    return math.e ** (-step * k)

# Get the data

In [10]:
transform = [torchvision.transforms.ToTensor(), torchvision.transforms.Normalize((0.1307,), (0.3081,))]
transform.append(torchvision.transforms.Lambda(torch.flatten))
mnist_train = torchvision.datasets.MNIST(
            "./data",
            train=True,
            download=True,
            transform=torchvision.transforms.Compose(transform),
)
mnist_test = torchvision.datasets.MNIST(
            "./data",
            train=False,
            download=True,
            transform=torchvision.transforms.Compose(transform),
        )
input_size = mnist_train.data.shape[1] * mnist_train.data.shape[2]
output_size = len(mnist_train.classes)

train_dataLoader = torch.utils.data.DataLoader(mnist_train, batch_size=BATCH_SIZE,shuffle=SHUFFLE)
test_dataLoader = torch.utils.data.DataLoader(mnist_test,batch_size=BATCH_SIZE,shuffle=SHUFFLE)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 9912422/9912422 [00:00<00:00, 18214150.61it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 28881/28881 [00:00<00:00, 498355.19it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 1648877/1648877 [00:00<00:00, 4488614.66it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████| 4542/4542 [00:00<00:00, 10892240.58it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



# Train the model

In [11]:
def train_model_fwrd(train_loader, test_loader, total_epochs, input_size, output_size):
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  with torch.no_grad():
        model = NeuralNet(input_size=input_size,output_size=output_size, hidden_sizes=[1024, 1024])
        model.to(device)
        model.float()
        model.train()

        optimizer: torch.optim.Optimizer = torch.optim.SGD(params=model.parameters(), lr=2e-4, nesterov=False, momentum=0.0, weight_decay=0.0)
        optimizer.zero_grad(set_to_none=True)

        scheduler: torch.optim.lr_scheduler._LRScheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=exponential_lr_decay)

        named_buffers = dict(model.named_buffers())
        named_params = dict(model.named_parameters())
        names = named_params.keys()
        params = named_params.values()

        base_model = copy.deepcopy(model)
        base_model.to("meta")

        # Train
        steps = 0
        t_total = 0.0

        for epoch in range(total_epochs):
            t0 = time.perf_counter()
            with tqdm(total=len(train_loader)) as pbar:
              for batch in train_loader:
                  pbar.update(1)
                  steps += 1
                  images, labels = batch

                  # Sample perturbation (tangent) vectors for every parameter of the model
                  v_params = tuple([torch.randn_like(p) for p in params])
                  f = partial(
                      functional_cross_entropy,
                      model=base_model,
                      names=names,
                      buffers=named_buffers,
                      x=images.to(device),
                      t=labels.to(device),
                  )

                  # Forward AD
                  loss, jvp = fc.jvp(f, (tuple(params),), (v_params,))

                  # Setting gradients
                  for v, p in zip(v_params, params):
                      p.grad = v * jvp

                  # Optimizer step
                  optimizer.step()

                  # Lr scaling
                  scheduler.step()

                  # Zero out grads
                  optimizer.zero_grad(set_to_none=True)

            t1 = time.perf_counter()
            t_total += t1 - t0
            print("Time/batch_time", t1 - t0, steps)
            print("Time/sps", steps / t_total, steps)

            acc = 0
            for batch in test_loader:
                images, labels = batch
                out = fc.functional_call(base_model, (named_params, named_buffers), (images.to(device),))
                pred = F.softmax(out, dim=-1).argmax(dim=-1)
                acc += (pred == labels.to(device)).sum()
            print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {loss.item():.4f}, Time (s): {t1 - t0:.4f}, Test accuracy: {(acc / len(mnist_test)).item():.4f}")
        print(f"Mean time: {t_total / total_epochs:.4f}")

        # Test
        acc = 0
        for batch in test_loader:
            images, labels = batch
            out = fc.functional_call(base_model, (named_params, named_buffers), (images.to(device),))
            pred = F.softmax(out, dim=-1).argmax(dim=-1)
            acc += (pred == labels.to(device)).sum()
        print("Test/accuracy", acc / len(mnist_test), steps)
        print(f"Test accuracy: {(acc / len(mnist_test)).item():.4f}")

In [12]:
def train_model_bkrd(train_loader, test_loader, total_epochs, input_size, output_size):
  use_cuda = torch.cuda.is_available()
  device = torch.device("cuda:0" if use_cuda else "cpu")

  model = NeuralNet(input_size=input_size,output_size=output_size, hidden_sizes=[1024, 1024])
  model.to(device)
  model.float()
  model.train()
  params = model.parameters()

  optimizer: torch.optim.Optimizer = torch.optim.SGD(params=model.parameters(), lr=2e-4, nesterov=False, momentum=0.0, weight_decay=0.0)
  optimizer.zero_grad(set_to_none=True)

  scheduler: torch.optim.lr_scheduler._LRScheduler = torch.optim.lr_scheduler.LambdaLR(optimizer=optimizer, lr_lambda=exponential_lr_decay)


  steps = 0
  t_total = 0.0
  for epoch in range(total_epochs):
      t0 = time.perf_counter()
      with tqdm(total=len(train_loader)) as pbar:
        for batch in train_loader:
            pbar.update(1)
            steps += 1
            images, labels = batch
            loss = cross_entropy(model, images.to(device), labels.to(device))
            loss.backward()

            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)

      t1 = time.perf_counter()
      t_total += t1 - t0
      print("Time/batch_time", t1 - t0, steps)
      print("Time/sps", steps / t_total, steps)
      # Test
      acc = 0
      for batch in test_loader:
          images, labels = batch
          out = model(images.to(device))
          pred = F.softmax(out, dim=-1).argmax(dim=-1)
          acc += (pred == labels.to(device)).sum()
      print(f"Epoch [{epoch+1}/{total_epochs}], Loss: {loss.item():.4f}, Time (s): {t1 - t0:.4f}, Test accuracy: {(acc / len(mnist_test)).item():.4f}")
  print("Mean time:", t_total / total_epochs)

    # Test
  acc = 0
  for batch in test_loader:
      images, labels = batch
      out = model(images.to(device))
      pred = F.softmax(out, dim=-1).argmax(dim=-1)
      acc += (pred == labels.to(device)).sum()
  print("Test/accuracy", acc / len(mnist_test), steps)
  print(f"Test accuracy: {(acc / len(mnist_test)).item():.4f}")



# Train Backward

In [13]:
train_model_bkrd(train_dataLoader, test_dataLoader, total_epochs=EPOCHS, input_size=input_size, output_size=output_size)

100%|██████████| 938/938 [00:19<00:00, 47.35it/s]


Time/batch_time 19.816051845999993 938
Time/sps 47.33536262872373 938
Epoch [1/50], Loss: 2.1495, Time (s): 19.8161, Test accuracy: 0.5586


100%|██████████| 938/938 [00:15<00:00, 59.29it/s]


Time/batch_time 15.826198306999999 1876
Time/sps 52.63416288104633 1876
Epoch [2/50], Loss: 2.0692, Time (s): 15.8262, Test accuracy: 0.6593


100%|██████████| 938/938 [00:14<00:00, 62.81it/s]


...

Time/batch_time 14.915769769999997 45024
Time/sps 61.945166223147815 45024
Epoch [48/50], Loss: 0.6883, Time (s): 14.9158, Test accuracy: 0.8505


100%|██████████| 938/938 [00:14<00:00, 62.62it/s]


Time/batch_time 14.98428550799997 45962
Time/sps 61.95837149499787 45962
Epoch [49/50], Loss: 0.8356, Time (s): 14.9843, Test accuracy: 0.8506


100%|██████████| 938/938 [00:15<00:00, 60.56it/s]


Time/batch_time 15.49578838299999 46900
Time/sps 61.929197675644794 46900
Epoch [50/50], Loss: 0.9392, Time (s): 15.4958, Test accuracy: 0.8506
Mean time: 15.146328956380005
Test/accuracy tensor(0.8506, device='cuda:0') 46900
Test accuracy: 0.8506


# Train Forward

In [14]:
train_model_fwrd(train_dataLoader, test_dataLoader, total_epochs=EPOCHS, input_size=input_size, output_size=output_size)

100%|██████████| 938/938 [00:16<00:00, 58.11it/s]


Time/batch_time 16.14793789600003 938
Time/sps 58.08791227964469 938
Epoch [1/50], Loss: 2.1591, Time (s): 16.1479, Test accuracy: 0.5174


100%|██████████| 938/938 [00:15<00:00, 59.91it/s]


Time/batch_time 15.662818399999878 1876
Time/sps 58.973762916661634 1876
Epoch [2/50], Loss: 2.0900, Time (s): 15.6628, Test accuracy: 0.6135


100%|██████████| 938/938 [00:15<00:00, 59.92it/s]


...

Time/batch_time 15.633139947000018 45024
Time/sps 59.98638345244146 45024
Epoch [48/50], Loss: 0.5864, Time (s): 15.6331, Test accuracy: 0.8760


100%|██████████| 938/938 [00:15<00:00, 61.03it/s]


Time/batch_time 15.374486561999902 45962
Time/sps 60.006933436249824 45962
Epoch [49/50], Loss: 0.4578, Time (s): 15.3745, Test accuracy: 0.8756


100%|██████████| 938/938 [00:15<00:00, 61.28it/s]


Time/batch_time 15.312666157999956 46900
Time/sps 60.03142454372058 46900
Epoch [50/50], Loss: 0.4153, Time (s): 15.3127, Test accuracy: 0.8760
Mean time: 15.6251
Test/accuracy tensor(0.8760, device='cuda:0') 46900
Test accuracy: 0.8760
